<a href="https://colab.research.google.com/github/Sopralapanca/CHL-clinical-data-encoder/blob/master/%5BMedium%5Dllm_semantic_routing_ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

una delle sfide più impegnative al momento è quella di avere un modello che fa semantic routing -> usiamo un llm

nella prima parte vediamo come strutturare il progetto con un semplce LLM
nella seconda parte utilizzeremo un rag

disclaimer i risultati potrebbero essere diversi in quanto il modello puo dare altre cose in ouput. cercare di essere più precisi possibili nei prompt

# Disclaimer

Write disclaimer

# Setup

Downloading Ollama

In [1]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13269    0 13269    0     0  47878      0 --:--:-- --:--:-- --:--:-- 47902
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
!ollama serve > server.log 2>&1 & # ollama will run by default at 127.0.0.1:11434

In [3]:
%%capture
agents_model = "qwen2.5:1.5b"
!ollama pull {agents_model}

In [4]:
%%capture
router_model = "qwen2.5:3b"
!ollama pull {router_model}

In [5]:
!ollama ls

NAME            ID              SIZE      MODIFIED               
qwen2.5:3b      357c53fb659c    1.9 GB    Less than a second ago    
qwen2.5:1.5b    65ec06548149    986 MB    20 seconds ago            


Serving the models at different ports

In [6]:
import os

In [7]:
os.environ['OLLAMA_HOST'] = "127.0.0.1:11438"
!ollama serve > server.log 2>&1 &

In [8]:
os.environ['OLLAMA_HOST'] = "127.0.0.1:11439"
!ollama serve > server.log 2>&1 &

Install required packages

In [9]:
%%capture
!pip install openai==1.57.4 ollama==0.4.4 pydantic==2.10.3

# Tools definition

In [10]:
from pydantic import BaseModel
from openai import OpenAI
import openai

summarizer_tool = {
    "type": "function",
    "function": {
        "name": "get_summarization",
        "description": "Call this method whenever you need to perform a summarization task, for example when a user asks 'Summarize this paragraph'. This method does not accepts input parameters.",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": [],
            "additionalProperties": False
        }
    }
}

topics_tool = {
    "type": "function",
    "function": {
        "name": "get_topics",
      "description": "Call this whenever you need to perform a topic extraction extraction task, for example when a user asks 'What are the topics of this paragraph?'. This method does not accepts input parameters.",
      "parameters": {
          "type": "object",
          "properties": {},
          "required": [],
          "additionalProperties": False
      }
    }
}


today_tool = {
    "type": "function",
    "function": {
        "name": "get_current_day",
      "description": "Get the current day of the week. Call this whenever you need to know what day it is today, for example when the user asks 'What day is it?'. This method does not accepts input parameters.",
      "parameters": {
          "type": "object",
          "properties": {},
          "required": [],
          "additionalProperties": False
      }
    }
}


tools = [topics_tool, summarizer_tool, today_tool]



# Agents

In [11]:
from openai import OpenAI

summarizer_client = OpenAI(
    base_url = 'http://localhost:11439/v1',
    api_key='ollama', # required, but unused
)

general_client = OpenAI(
    base_url = 'http://localhost:11438/v1',
    api_key='ollama', # required, but unused
)

In [12]:
def build_history_agents(messages: list[dict[str, str]], sys_prompt: str):
    history = [{"role":"system", "content": sys_prompt}]

    for message in messages:
      if message["role"] != "system":
        history.append(message)

    return history

def get_summarization(history: list[dict[str, str]]):
  print("calling summarization model")

  sys_prompt = "You are a summarizer agent. Your only role is to summarize user message. When user asks for other action or want to chit-chat just answer 'Pass.'"

  response = summarizer_client.chat.completions.create(
    model=agents_model,
    messages=build_history_agents(history, sys_prompt),
  )

  return response.choices[0].message.content

def get_topics(history: list[dict[str, str]]):
  print("calling topic model")
  sys_prompt = "You are a topic extractor model. Your role is to examinie user message and extract the topics. When user asks for other action or want to chit-chat just answer 'Pass.'"

  response = general_client.chat.completions.create(
    model=agents_model,
    messages=build_history_agents(history, sys_prompt),
  )

  return response.choices[0].message.content

import datetime
def get_current_day():
  print("calling today model")

  return datetime.datetime.now().strftime("%A")


# Utils

In [13]:
import json
import re

def execute_function_call(response, history):
  if response.choices[0].finish_reason == "tool_calls":
    tool_call = response.choices[0].message.tool_calls[0]
    function_name = tool_call.function.name

    if function_name == "get_current_day":
      result = globals()[function_name]()
    else:
      result = globals()[function_name](history)


    function_call_result_message = {
      "role": "tool",
      "content": json.dumps({
          "result": result
      }),
      "tool_call_id": response.choices[0].message.tool_calls[0].id
    }

    response_dict = response.model_dump()
    response_dict["choices"][0]["message"]

    history = history + [response_dict["choices"][0]["message"]] + [function_call_result_message]

    return history

In [14]:
def build_router_history(messages: list[dict[str, str]], response):
    res = response.choices[0].message.content
    messages.append(
        {
            "role": "assistant",
            "content": res
        }
    )
    print(res)
    return messages

# Router

In [15]:
router_client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)


def call_router(messages: list[dict[str, str]], use_tools=True):

  response = router_client.chat.completions.create(
    model=router_model,
    messages=messages,
    tools=tools if use_tools else None
  )

  return response

# Chat with the assistant

In [16]:
sys_prompt = "You are a routing model. Your role is to understand the chat history and user intent and eventualy to redirect the messages to the correct model that will handle the conversation. Your are allowed to chat with the user but only for chit-chatting, otherwise if you are asked to perform specific tasks just call other functions."

messages = [
  {
      "role": "system",
      "content": sys_prompt
  },
  {
      "role": "user",
      "content": "Hi, can you summarize this paragraph? The red glow of tail lights indicating another long drive home from work after an even longer 24-hour shift at the hospital. The shift hadn’t been horrible but the constant stream of patients entering the ER meant there was no downtime. She had some of the “regulars” in tonight with new ailments they were sure were going to kill them. It’s amazing what a couple of Tylenol and a physical exam from the doctor did to eliminate their pain, nausea, headache, or whatever other mild symptoms they had. Sometimes she wondered if all they really needed was some interaction with others and a bit of the individual attention they received from the nurses."
  }
]

response = call_router(messages)
response

ChatCompletion(id='chatcmpl-4', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_96uqu57o', function=Function(arguments='{"text":"The red glow of tail lights indicating another long drive home from work after an even longer 24-hour shift at the hospital. The shift hadn’t been horrible but the constant stream of patients entering the ER meant there was no downtime. She had some of the \'regulars\' in tonight with new ailments they were sure were going to kill them. It\'s amazing what a couple of Tylenol and a physical exam from the doctor did to eliminate their pain, nausea, headache, or whatever other mild symptoms they had. Sometimes she wondered if all they really needed was some interaction with others and a bit of the individual attention they received from the nurses."}', name='get_summarization'), type='fun

In [17]:
messages = execute_function_call(response, messages)

response = call_router(messages, use_tools=False)

messagges = build_router_history(messages, response)


calling summarization model
The summary of the paragraph is:

A person reflects on their 24-hour shift at the hospital, noting that while it hasn't been terrible overall—constantly dealing with patient influxes without downtime—they often have repeat patients who come back after various minor issues (such as pain and nausea). These issues are typically resolved through medication (Tylenol) along with physical examinations from doctors. The person reflects on how the shifts remind them of how simple interactions and more personalized care can resolve many minor problems associated with being alone in the Emergency Room all day long.


In [18]:
messages.append({
      "role": "user",
      "content": "Now I want to know what day is it"
  }
)

response = call_router(messages)
print(response)

ChatCompletion(id='chatcmpl-650', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ppy9ohoc', function=Function(arguments='{}', name='get_current_day'), type='function', index=0)]))], created=1735233249, model='qwen2.5:3b', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=17, prompt_tokens=757, total_tokens=774, completion_tokens_details=None, prompt_tokens_details=None))


In [19]:
messages = execute_function_call(response, messages)

response = call_router(messages, use_tools=False)
messagges = build_router_history(messages, response)

calling today model
Today is Thursday.


In [20]:
messages.append({
      "role": "user",
      "content": "Now can you extract topics from the paragraph that I sent you on the first message?"
  }
)

response = call_router(messages)
print(response)

ChatCompletion(id='chatcmpl-407', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_mzpgu1fu', function=Function(arguments='{}', name='get_topics'), type='function', index=0)]))], created=1735233251, model='qwen2.5:3b', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=16, prompt_tokens=812, total_tokens=828, completion_tokens_details=None, prompt_tokens_details=None))


In [21]:
messages = execute_function_call(response, messages)

response = call_router(messages, use_tools=False)
messagges = build_router_history(messages, response)

calling topic model
The main topics from the paragraph are:

1. A 24-hour shift at the hospital's Emergency Room (ER)
2. The constant influx of patients during the shift
3. The treatments provided (Tylenol, physical exams)
4. Repeat patients with various minor issues
5. Reflection on personal care and interactions needed for better patient outcomes


In [22]:
messages.append({
      "role": "user",
      "content": "Thank you!! You are a very helpful assistant. Now, say hello to our Medium friends :)"
  }
)

response = call_router(messages)
messagges = build_router_history(messages, response)

Hello to our Medium friends! How can I assist you further?


# Check the history

In [23]:
messages

[{'role': 'system',
  'content': 'You are a routing model. Your role is to understand the chat history and user intent and eventualy to redirect the messages to the correct model that will handle the conversation. Your are allowed to chat with the user but only for chit-chatting, otherwise if you are asked to perform specific tasks just call other functions.'},
 {'role': 'user',
  'content': 'Hi, can you summarize this paragraph? The red glow of tail lights indicating another long drive home from work after an even longer 24-hour shift at the hospital. The shift hadn’t been horrible but the constant stream of patients entering the ER meant there was no downtime. She had some of the “regulars” in tonight with new ailments they were sure were going to kill them. It’s amazing what a couple of Tylenol and a physical exam from the doctor did to eliminate their pain, nausea, headache, or whatever other mild symptoms they had. Sometimes she wondered if all they really needed was some interact